# `00-bill-import` - Importing Bill Data

In this notebook we import data about Bills connected to public lands using the ProPublica Congress API.

![../img/data_model.png](../img/data_model.png)

## TODO:

- [x] search for bills containing name of a public land
- [x] find sponsor and cosponsors of each bill
- [x] find subjects of each bill
- [ ] privately funded legislator trips


Other options include: bill actions, related bills

In [ ]:
from neo4j import GraphDatabase
import requests
import time
import pprint
import json
import os

In [ ]:
pp = pprint.PrettyPrinter(indent=4)

In [ ]:
# DEFINE the following environment variables:

NEO4J_URI           = "neo4j://localhost:7687"
NEO4J_USER          = "neo4j"
NEO4J_PASSWORD      = "letmeinnow"
NEO4J_DATABASE      = "neo4j"
PROPUBLICA_API_KEY  = os.environ.get("PROPUBLICA_API_KEY")
PROPUBLICA_ENDPOINT = 'https://api.propublica.org/congress/v1/bills/search.json?query="{name}"'

driver = GraphDatabase.driver(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

In [ ]:
# TODO: download PP geojsons and load into Neo4j

## Find Bills

Now we use the ProPublica Congress API to search for any bills that are related to federally managed protected lands. 

TODO: 

- [ ] add arrows data model diagram
- [ ] include Congress number
- [ ] include committees



In [ ]:
# TODO: query Neo4j for parcel names

# Let's start with only federally owned parcels for now
PARCEL_NAME_QUERY = """
MATCH (p:Parcel) WHERE p.Own_Type = "FED"
RETURN p.name AS name
"""

IMPORT_BILL_QUERY = """
WITH $bill AS bill
MATCH (p:Parcel {name: $name})
MERGE (b:Bill {billId: bill.bill_id})
SET b.number          = bill.number,
    b.title           = bill.title,
    b.short_title     = bill.short_title,
    b.introduced_date = Date(bill.introduced_date),
    b.url             = bill.congressdotgov_url
MERGE (l:Legislator {legislatorId: bill.sponsor_id})
ON CREATE SET
    l.name = bill.sponsor_name,
    l.state = bill.sponsor_state,
    l.party = bill.sponsor_party
MERGE (c:Committee {name: bill.committees})
MERGE (b)<-[:SPONSORED]-(l)
MERGE (b)-[:LEGISLATES]->(p)
MERGE (b)-[:IN_COMMITTEE]->(c)
"""

# TODO: cosponsors ==> legislative influence

In [ ]:
def get_parcel_names(tx):
    parcels = []
    result = tx.run(PARCEL_NAME_QUERY)
    for record in result:
        parcels.append(record["name"])
    return parcels


def write_bills(tx, bill, parcel):
    result = tx.run(IMPORT_BILL_QUERY, bill=bill, name=parcel)
    return result

In [ ]:

# Fetch all parcel names from Neo4j
with driver.session(database=NEO4J_DATABASE) as session:
    parcels = session.execute_read(get_parcel_names)
    print(parcels)


In [ ]:
# TODO: Query Propublica API

headers = {'X-API-Key': PROPUBLICA_API_KEY}

for parcel in parcels:
    try:
        r = requests.get(PROPUBLICA_ENDPOINT.format(name=parcel), headers=headers)
        response = r.json()
        #print(response)
        for result in response['results']:
            for bill in result['bills']:
                with driver.session(database=NEO4J_DATABASE) as session:
                    session.execute_write(write_bills, bill, parcel)
        print(parcel)
        time.sleep(3)
    except:
        pass

## Bill Cosponsors

### TODO:



In [ ]:
PROPUBLICA_COSPONSOR_URL = 'https://api.propublica.org/congress/v1/{congress}/bills/{billid}/cosponsors.json'

In [ ]:
# Get all bills query

# For each bill, query propublica API to find cosponsors

# insert into neo4j

```
"cosponsors":[
 {
   "cosponsor_id": "J000032",
   "name": "Sheila  Jackson Lee",
   "cosponsor_title": "Rep.",
   "cosponsor_state": "TX",
   "cosponsor_party": "D",
   "cosponsor_uri": "https://api.propublica.org/congress/v1/members/J000032.json",
   "date": "2016-01-25"
 },
```

In [ ]:
GET_BILLS_QUERY = """
MATCH (b:Bill)
RETURN b.billId AS bill
"""

WRITE_COSPONSOR_QUERY = """
MATCH (b:Bill {billId: $billId})
UNWIND $cosponsors AS co
MERGE (l:Legislator {legislatorId: co.cosponsor_id})
SET  l.name = co.name,
    l.state = co.cosponsor_state,
    l.party = co.cosponsor_party,
    l.title = co.cosponsor_title
MERGE (l)-[r:COSPONSORED]->(b)
SET r.date = Date(co.date)
RETURN COUNT(*) AS num_created
"""

In [ ]:
def get_bills(tx):
    bills = []
    result = tx.run(GET_BILLS_QUERY)
    for record in result:
        bills.append(record["bill"])
    return bills


def write_cosponsor(tx, cosponsors, bill):
    result = tx.run(WRITE_COSPONSOR_QUERY, cosponsors=cosponsors, billId=bill)
    return result

In [ ]:
with driver.session(database=NEO4J_DATABASE) as session:
    bills = session.execute_read(get_bills)
    print(bills)

In [ ]:
headers = {'X-API-Key': PROPUBLICA_API_KEY}

for bill in bills:
    try:
        print(PROPUBLICA_COSPONSOR_URL.format(congress=bill[-3:], billid=bill))
        print(bill[-3:])
        print(bill[:-4])
        r = requests.get(PROPUBLICA_COSPONSOR_URL.format(congress=bill[-3:], billid=bill[:-4]), headers=headers)
        response = r.json()
        print(response)
        for result in response['results']:
            with driver.session(database=NEO4J_DATABASE) as session:
                res = session.execute_write(write_cosponsor, result['cosponsors'], bill)
                print(res)
        #print(parcel)
        time.sleep(3)
    except:
        pass

## Bill Subjects

### TODO:



In [ ]:
PROPUBLICA_SUBJECTS_URL = 'https://api.propublica.org/congress/v1/{congress}/bills/{billid}/subjects.json'

Example response:

```
"results":[
      {
         "congress": "115",
         "bill_id": "hr2810-115",
         "bill_slug": "hr2810",
         "bill_type": "hr",
         "number": "H.R.2810",
         "bill_uri": "https://api.propublica.org/congress/v1/115/bills/hr2810.json",
         "url_number": "hr2810",
         "title": "To authorize appropriations for fiscal year 2018 for military activities of the Department of Defense and for military construction, to prescribe military personnel strengths for such fiscal year, and for other purposes.",
         "sponsor_title": "Rep.",
         "sponsor_id": "T000238",
         "sponsor_name": "William M. Thornberry",
         "sponsor_state": "TX",
         "sponsor_party": "R",
         "sponsor_uri": "https://api.propublica.org/congress/v1/members/T000238.json",
         "introduced_date": "2017-06-07",
         "number_of_cosponsors": 1,
         "committees": "House Transportation and Infrastructure Committee",
         "latest_major_action_date": "2017-12-12",
         "latest_major_action": "Became Public Law No: 115-91.",
         "house_passage_vote": "2017-10-12",
         "senate_passage_vote": "2017-09-18",
                  "subjects":[
         {
           "name": "Congressional oversight",
           "url_name": "congressional-oversight"
         },
                  {
           "name": "Europe",
           "url_name": "europe"
         },
```

In [ ]:
GET_BILLS_QUERY = """
MATCH (b:Bill)
RETURN b.billId AS bill
"""

WRITE_SUBJECTS_QUERY = """
MATCH (b:Bill {billId: $billId})
UNWIND $subjects AS sub
MERGE (s:Subject {slug: sub.url_name})
ON CREATE SET s.name = sub.name
MERGE (b)-[:HAS_SUBJECT]->(s)
RETURN COUNT(*) AS num
"""

In [ ]:
def get_bills(tx):
    bills = []
    result = tx.run(GET_BILLS_QUERY)
    for record in result:
        bills.append(record["bill"])
    return bills


def write_subjects(tx, subjects, bill):
    result = tx.run(WRITE_SUBJECTS_QUERY, subjects=subjects, billId=bill)
    return result

In [ ]:
with driver.session(database=NEO4J_DATABASE) as session:
    bills = session.execute_read(get_bills)
    print(bills)

In [ ]:
headers = {'X-API-Key': PROPUBLICA_API_KEY}

for bill in bills:
    try:
        print(PROPUBLICA_SUBJECTS_URL.format(congress=bill[-3:], billid=bill[:-4]))
        print(bill[-3:])
        print(bill[:-4])
        r = requests.get(PROPUBLICA_SUBJECTS_URL.format(congress=bill[-3:], billid=bill[:-4]), headers=headers)
        response = r.json()
        print(response)
        for result in response['results']:
            with driver.session(database=NEO4J_DATABASE) as session:
                res = session.execute_write(write_subjects, result['subjects'], bill)
                print(res)
        #print(parcel)
        time.sleep(3)
    except Exception as e:
        print(e)
        pass

## Legislator Trips

Privately funded trips

### TODO

- [ ] handle pagination

Example result:

```
{
    "status": "OK",
    "copyright": "Copyright (c) 2019 Pro Publica Inc. All Rights Reserved.",
    "num_results": 43,
    "offset": 0,
    "member_id": "W000797",
    "api_uri": "https://api.propublica.org/congress/v1/members/W000797.json",
    "display_name": "Debbie Wasserman Schultz",
    "results": [{
        "filing_type": "Original",
        "traveler": "Lauren Wolman",
        "is_member": 0,
        "congress": 116,
        "departure_date": "2019-11-02",
        "return_date": "2019-11-10",
        "chamber": "House",
        "destination": "Jerusalem, Israel",
        "sponsor": "American Israel Education Foundation",
        "document_id": "500022599",
        "pdf_url": "http://clerk.house.gov/GTImages/MT/2019/500022599.pdf"
    }, {
        "filing_type": "Original",
        "traveler": "Lauren Wolman",
        "is_member": 0,
        "congress": 116,
        "departure_date": "2019-11-02",
        "return_date": "2019-11-10",
        "chamber": "House",
        "destination": "Tel Aviv, Israel",
        "sponsor": "American Israel Education Foundation",
        "document_id": "500022599",
        "pdf_url": "http://clerk.house.gov/GTImages/MT/2019/500022599.pdf"
    }, 
```

In [ ]:
PROPUBLICA_TRIPS_URL='https://api.propublica.org/congress/v1/members/{memberid}/private-trips.json'

In [ ]:
GET_LEGISLATORS_QUERY = """
MATCH (l:Legislator)
RETURN l.legislatorId AS member
"""

# $memberId, $trips
WRITE_TRIPS_QUERY = """
MATCH (l:Legislator {legislatorId: $memberId})
UNWIND $trips AS trip
MERGE (t:Trip {documentId: trip.document_id})
ON CREATE SET t.url = trip.pdf_url,
        t.departureDate = Date(trip.departure_date),
        t.returnDate = Date(trip.return_date)
MERGE (o:Organization {name: trip.sponsor})
MERGE (d:Destination {name: trip.destination})
MERGE (l)-[:HAS_PRIVATE_TRIP]->(t)
MERGE (t)-[:FUNDED_BY]->(o)
MERGE (t)-[:DESTINATION]->(d)
RETURN COUNT(*) AS num
"""

In [ ]:
def get_legislators(tx):
    members = []
    result = tx.run(GET_LEGISLATORS_QUERY)
    for record in result:
        members.append(record["member"])
    return members


def write_trips(tx, trips, member):
    result = tx.run(WRITE_TRIPS_QUERY, trips=trips, memberId=member)
    return result

In [ ]:
with driver.session(database=NEO4J_DATABASE) as session:
    members = session.execute_read(get_legislators)
    print(members)

In [ ]:
headers = {'X-API-Key': PROPUBLICA_API_KEY}

for member in members:
    try:
        print(PROPUBLICA_TRIPS_URL.format(memberid=member))
        r = requests.get(PROPUBLICA_TRIPS_URL.format(memberid=member), headers=headers)
        response = r.json()
        print(response)
        trips = response['results']
        with driver.session(database=NEO4J_DATABASE) as session:
            res = session.execute_write(write_trips, trips, member)
            print(res)
        time.sleep(3)
    except Exception as e:
        print(e)
        pass